# Create basic network
Based on BGT data

In [ ]:
# Select where to run notebook: "azure" or "local"
my_run = "azure"

In [ ]:
import set_path

import numpy as np
import pandas as pd

import shapely.ops as so
import geopandas as gpd
from geopandas import GeoDataFrame
from centerline.geometry import Centerline

from tqdm.notebook import tqdm_notebook
tqdm_notebook.pandas()

import poly_utils
import bgt_utils

import matplotlib.pyplot as plt
import folium

import settings as st
if my_run == "azure":
    import config_azure as cf
elif my_run == "local":
    import config as cf

In [ ]:
#os.system('sudo blobfuse /home/azureuser/cloudfiles/code/blobfuse/sidewalk --tmp-path=/mnt/resource/blobfusetmp --config-file=/home/azureuser/cloudfiles/code/blobfuse/fuse_connection_sidewalk.cfg -o attr_timeout=3600 -o entry_timeout=3600 -o negative_timeout=3600 -o allow_other -o nonempty')

## Import BGT data

In [ ]:
# Get BGT data
df_bgt_full = bgt_utils.get_bgt_data_for_bbox(st.bbox, st.bgt_layers)
df_bgt_full['naam'].value_counts()

## Select pilot areas

In [ ]:
# Import areas
df_areas = gpd.read_file(cf.output_pilot_area)

In [ ]:
# Only keep BGT data within pilot areas
df_bgt = df_bgt_full.sjoin(df_areas, how='inner', predicate='within')  # note: only sidewalk polygons fully inside area are included
df_bgt['naam_left'].value_counts()

### Visualize

In [ ]:
# Set center of map
area_coords = [52.375664816888225, 4.8632280955697995]

# Set to map coordinate reference system
df_bgt_show = df_bgt.to_crs("EPSG:4326")

# Create basic map
my_map = folium.Map(
    location=area_coords,
    tiles='cartodbpositron',
    min_zoom=10, max_zoom=20, zoom_start=13,
    zoom_control=True, control_scale=True, control=False
    )

## Add sidewalks
#for _, r in df_bgt_show[df_bgt_show['naam_left'] == 'voetpad'].iterrows():
#    sim_geo = gpd.GeoSeries(r["geometry"]) #.simplify(tolerance=0.0001)
#    geo_j = sim_geo.to_json()
#    geo_j = folium.GeoJson(data=geo_j, style_function=lambda x: {"fillColor": "orange"})
#    geo_j.add_to(my_map)

## Add sidewalks
#for _, r in df_bgt_show[df_bgt_show['naam_left'] == 'inrit'].iterrows():
#    sim_geo = gpd.GeoSeries(r["geometry"]) #.simplify(tolerance=0.0001)
#    geo_j = sim_geo.to_json()
#    geo_j = folium.GeoJson(data=geo_j, style_function=lambda x: {"fillColor": "red"})
#    geo_j.add_to(my_map)

# Add sidewalks
for _, r in df_bgt_show[df_bgt_show['naam_left'] == 'voetgangersgebied'].iterrows():
    sim_geo = gpd.GeoSeries(r["geometry"]) #.simplify(tolerance=0.0001)
    geo_j = sim_geo.to_json()
    geo_j = folium.GeoJson(data=geo_j, style_function=lambda x: {"fillColor": "green"})
    geo_j.add_to(my_map)

my_map

## Pre-process BGT data

In [ ]:
## Remove obstacles (small interiors)
df_bgt['geometry_no_holes'] = df_bgt.progress_apply(  
   lambda row: poly_utils.remove_interiors(row.geometry, st.min_interior_size), axis=1)
df_bgt = df_bgt.set_geometry('geometry_no_holes')

# Merge sidewalk polygons
df_bgt = GeoDataFrame(geometry=gpd.GeoSeries(df_bgt['geometry_no_holes'].unary_union))
df_bgt = gpd.GeoDataFrame(df_bgt.geometry.explode()) 

# Ignore sidewalk polygons that are too small
df_bgt['area'] = df_bgt['geometry'].area
df_bgt = df_bgt[df_bgt.area > st.min_area_size]

## Create centerlines

In [ ]:
# Calculate centerlines
df_bgt['centerlines'] = df_bgt.progress_apply(
   lambda row: Centerline(row.geometry, interpolation_distance=0.5), axis=1)
df_bgt = df_bgt.set_geometry('centerlines')

In [ ]:
# Merge lines
df_bgt['centerlines'] = df_bgt['centerlines'].progress_apply(so.linemerge)

# Add sidewalk ID
df_bgt['sidewalk_id'] = range(0, len(df_bgt))

# Get each centerline in separate row
df_bgt_exp = df_bgt.explode(ignore_index=True)
df_bgt_exp.reset_index(inplace=True, names='cl_id')

## Cut lines that are too long

In [ ]:
# Remove geometry column and add length column
df_bgt_cut = df_bgt_exp.drop('geometry', axis=1)
df_bgt_cut['length'] = df_bgt_cut['centerlines'].length

# Cut linestrings longer than a max length (this takes a while)
df_bgt_cut = poly_utils.shorten_linestrings(df_bgt_cut, st.max_ls_length)

# Add shortened linestrings to previous dataframe
df_bgt_exp = pd.merge(df_bgt_exp.drop(['centerlines'], axis=1), df_bgt_cut.drop(['area', 'sidewalk_id'], axis=1), 
                                                            how = 'outer', on = 'cl_id')
#del df_bgt_cut

## Store final output

In [ ]:
# Finalize dataframe
df_bgt_exp = df_bgt_exp[['centerlines', 'length', 'cl_id', 'sidewalk_id', 'area']]
df_bgt_exp = df_bgt_exp.set_crs(st.CRS)
df_bgt_exp.head(3)

In [ ]:
# Write basic network to file
df_bgt_exp.to_file(cf.output_basic_network, driver='GPKG')

### Visualize

In [ ]:
# Set to map coordinate reference system
df = df_bgt_exp.to_crs("EPSG:4326")
df_areas = df_areas.to_crs("EPSG:4326")
df_bgt = df_bgt.set_geometry('geometry').set_crs(st.CRS).to_crs("EPSG:4326")

In [ ]:
# Set center of map
area_coords = [52.375664816888225, 4.8632280955697995]

# Create basic map
my_map = folium.Map(
    location=area_coords,
    tiles='cartodbpositron',
    min_zoom=10, max_zoom=20, zoom_start=13,
    zoom_control=True, control_scale=True, control=False
    )

# Add wijken
for _, r in df_areas.iterrows():
    sim_geo = gpd.GeoSeries(r["geometry"])
    geo_j = sim_geo.to_json()
    geo_j = folium.GeoJson(data=geo_j, style_function=lambda x: {"fillColor": "orange"})
    geo_j.add_to(my_map)

# Add sidewalks
for _, r in df_bgt.iterrows():
    sim_geo = gpd.GeoSeries(r["geometry"])
    geo_j = sim_geo.to_json()
    geo_j = folium.GeoJson(data=geo_j, style_function=lambda x: {"fillColor": "grey"})
    geo_j.add_to(my_map)

# Add basic network
for _, r in df.iterrows():
    sim_geo = gpd.GeoSeries(r["centerlines"])
    geo_j = sim_geo.to_json()
    geo_j = folium.GeoJson(data=geo_j, style_function=lambda x: {"color": "black"})   # TODO random colors
    geo_j.add_to(my_map)

my_map

In [ ]:
# Add title
loc = 'Basic network'
title_html = '''
             <h3 align="center" style="font-size:16px"><b>{}</b></h3>
             '''.format(loc)

my_map.get_root().html.add_child(folium.Element(title_html))

In [ ]:
# Store map
my_map.save(cf.basic_network_map)